In [1]:
import pandas as pd
from sklearn.metrics import ndcg_score, precision_score, recall_score
from pathlib import Path
import math

In [2]:
def show_Hu(row):
    #wybieranie id filmów, dla których w zbiorze testowym użytkownik wystawił jakieś oceny
    nonzero = row[row > 0].index
    nonzero_indices = nonzero.values
    row['Hu'] = nonzero_indices.tolist()
    return row
def find_topN_items(users_ratings,N):
    topN = users_ratings.nlargest(N) #N najlepiej ocenianych filmów (rating i indeks) 
                                        #zwraca się od najwiekszej do najmn wartosci
    topN_indices = topN.index #indeksy tych najlepiej ocenianych
    #wyrzucam z topN filmy, dla których wystawiono zero
    top_nonzero_indices = users_ratings[topN_indices][users_ratings[topN_indices] > 0].index
    users_ratings["topN_movies"] = top_nonzero_indices.values
    return users_ratings
def NDCG(row,N):
    generated = row.loc['topN_movies']
    from_test_set = row.loc['Hu']
    #sprawdzenie który n-ty element z TopN został irl oceniony przez użytkownika
    available_n = [index + 1  for index, element in enumerate(generated) if element in from_test_set]
    DCG = sum([1 / math.log(1+n,2) for n in available_n])
    IDCG = sum([math.log(1+n,2) for n in range(1,N+1)])
    NDCG = DCG / IDCG
    row['NDCG'] = NDCG
    return row
def Precision_Recall(row,N):
    generated = row.loc['topN_movies']
    from_test_set = row.loc['Hu']
    intersection = sum([1 for element in generated if element in from_test_set])
    
    Precision = intersection * 100 / N
    row['Precision'] = Precision
    
    row['Recall'] = intersection * 100 / len(from_test_set)
    return row


In [ ]:

def nazwepozniej(test_matrix, train_matrix, alpha, N, ratings_matrix, test_users_cl, train_users_cl, items_cl):
    
    generated = create_matrix_for_generated(test_matrix, train_matrix, alpha, ratings_matrix, test_users_cl, train_users_cl, items_cl)
    test_with_Hu = test_matrix.apply(lambda row: show_Hu(row), axis = 1)
    generated_topN = generated.apply(lambda row: find_topN_items(row, N), axis = 1)
    Hu = test_with_Hu.iloc[:,[-1]].sort_index()
    
    merged = pd.merge(generated_topN, Hu, on='user_id', how = 'left')
    merged = merged.apply(lambda row:  NDCG(row,N),axis = 1)
    merged = merged.apply(lambda row:  Precision_Recall(row,N),axis = 1)
    return(merged)
"""output_final = nazwepozniej(test_set, train_set, 0.5, 50, matrix, users_test_clustered, users_train_clustered, items_clustered)
""""""hu1 = output_final[0]
generated1 = output_final[1]
test_with_hu1 = output_final[2]
generatetopN1 = output_final[3]
merged1 = output_final[4]"""

In [24]:
generated = pd.read_csv(Path('..','outputs','test_ratings.csv'))
generated_HP_for_clust = pd.read_csv(Path('test_mtrx_gen.csv'))
original = pd.read_csv(Path('test_mtrx_org.csv'))
#original = original.iloc[:,original.columns.isin(generated.columns)]
generated.shape, generated_HP_for_clust.shape, original.shape

((309015, 3), (189, 1636), (189, 1683))

In [25]:
original = original.sort_values('user_id')
original_no_user = original.iloc[:,1:]
org_np = original_no_user.iloc[:,1:].to_numpy()

to_rename = {str(old_name) : int(old_name) for old_name in original_no_user.columns}
to_rename

original_no_user = original_no_user.rename(columns=to_rename)
original_no_user.columns

Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
       ...
       1673, 1674, 1675, 1676, 1677, 1678, 1679, 1680, 1681, 1682],
      dtype='int64', length=1682)

In [26]:
original = original.set_index('user_id')
original = original.rename(columns=to_rename)

In [27]:
original

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
24,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
930,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
938,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
#original_matrix = original.pivot(index='user_id', columns='movie_id', values='rating')
#original_matrix = original_matrix.fillna(0)

generated_matrix = generated.pivot(index='user_id', columns='movie_id', values='rating')
generated_matrix = generated_matrix.fillna(0)

original_matrix = original_no_user.iloc[:,original_no_user.columns.isin((generated_matrix.columns))]
generated_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1667,1668,1669,1670,1671,1672,1673,1674,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
24,12.748544,16.550229,14.956472,18.810884,26.120023,16.480238,10.624590,18.131628,13.635854,17.852740,...,115.341457,115.341457,173.011569,115.341457,95.731259,0.000986,75.408967,0.000986,0.000740,0.00074
31,51.798935,107.153309,119.853436,83.162018,81.971747,65.201449,57.456515,60.426855,50.918587,56.793352,...,0.000740,0.000740,0.000493,0.000740,0.000247,237.266163,0.000740,137.827871,243.297604,0.00074
32,12.748544,16.550229,14.956472,18.810884,26.120023,16.480238,10.624590,18.131628,13.635854,17.852740,...,115.341457,115.341457,173.011569,115.341457,95.731259,0.000986,75.408967,0.000986,0.000740,0.00074
34,6.354915,5.207845,4.894923,10.940749,7.192495,5.730859,7.051564,7.295324,8.455044,11.165628,...,0.000740,0.000740,0.000493,0.000740,0.000247,0.000986,0.000740,0.000986,0.000740,0.00074
40,51.798935,107.153309,119.853436,83.162018,81.971747,65.201449,57.456515,60.426855,50.918587,56.793352,...,0.000740,0.000740,0.000493,0.000740,0.000247,237.266163,0.000740,137.827871,243.297604,0.00074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,51.798935,107.153309,119.853436,83.162018,81.971747,65.201449,57.456515,60.426855,50.918587,56.793352,...,0.000740,0.000740,0.000493,0.000740,0.000247,237.266163,0.000740,137.827871,243.297604,0.00074
930,12.748544,16.550229,14.956472,18.810884,26.120023,16.480238,10.624590,18.131628,13.635854,17.852740,...,115.341457,115.341457,173.011569,115.341457,95.731259,0.000986,75.408967,0.000986,0.000740,0.00074
938,12.748544,16.550229,14.956472,18.810884,26.120023,16.480238,10.624590,18.131628,13.635854,17.852740,...,115.341457,115.341457,173.011569,115.341457,95.731259,0.000986,75.408967,0.000986,0.000740,0.00074


In [34]:
type(original_np), type(generated_np)

(numpy.ndarray, numpy.ndarray)

macierze na których działam to generated_matrix i original_matrix

In [31]:
original_np = original_matrix.to_numpy()
generated_np = generated_matrix.to_numpy()

precision = precision_score(original_np.flatten(), generated_np.flatten())
print("Precision:", precision)

# Recall
recall = recall_score(original_np.flatten(), generated_np.flatten())
print("Recall:", recall)

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [38]:
import numpy as np
recommendation_matrix = np.array([[1, 0, 1, 0, 1],
                                  [0, 1, 1, 0, 0],
                                  [1, 1, 0, 1, 0],
                                  [0, 0, 1, 0, 1]])

original_matrix = np.array([[1, 0, 1, 0, 1],
                            [0, 1, 1, 1, 0],
                            [1, 0, 0, 1, 0],
                            [0, 1, 1, 0, 1]])

original_matrix = original_np
recommendation_matrix = generated_np
# Precision
precision = precision_score(original_matrix.flatten(), recommendation_matrix.flatten())
print("Precision:", precision)

# Recall
recall = recall_score(original_matrix.flatten(), recommendation_matrix.flatten())
print("Recall:", recall)

type(original_matrix)

ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [5]:
# Przykładowy słownik
my_dict = {'3':[]}

# Pętla iterująca
for i in range(5):
    key = '3'  # Przykładowa generacja klucza
    value = i * 2  # Przykładowa generacja wartości

    # Dodawanie wartości do słownika pod odpowiednim kluczem
    my_dict[key].append(value)

# Wyświetlanie rezultatu
print(my_dict)


{'3': [0, 2, 4, 6, 8]}


In [25]:
generated_matrix_mine = generated_matrix.copy()
generated_matrix_mine = original.apply(lambda row: show_Hu(row), axis = 1)
generated_matrix_topN = generated_matrix.apply(lambda row: find_topN_items(row, 20), axis = 1)

In [59]:
merged = pd.merge(generated_matrix_topN, generated_matrix_mine['Hu'], on='user_id', how='left')
merged = merged.apply(lambda row:  NDCG(row,20),axis = 1)


In [65]:
merged[merged['NDCG'] > 0]['NDCG']

user_id
87     0.005441
276    0.005909
389    0.003596
405    0.050016
449    0.004598
524    0.005091
535    0.003596
588    0.004128
705    0.003663
847    0.005909
894    0.005091
Name: NDCG, dtype: float64

In [84]:
generated_matrix.shape
original_small_isin = original.loc[:, original.columns.isin(generated_matrix.columns)]
original_small_isin
original_small = original.loc[:, generated_matrix.columns.values]

#sum(original_small == original_small_isin) == original_small_isin.shape[0] * original_small_isin.shape[1]

sum(original_small == original_small_isin), original_small_isin.shape[0] * original_small_isin.shape[1], original_small.shape[0] * original_small.shape[1]

(1340816, 309015, 309015)

In [89]:
original_small.equals(original_small_isin)
sum(original_small_isin.columns.values == original_small.columns.values) == len(original_small.columns.values)
sum(original_small_isin.index.values == original_small.index.values) == len(original_small.index.values)

True

In [91]:
ndcg_score(original_small, generated_matrix, k = 20)

0.00204697599657979

In [125]:
gen_np = generated_matrix.to_numpy()
org_np = original_matrix.to_numpy()

In [126]:
ndcg_at_k = ndcg_score(original_matrix, generated_matrix, k=20)
ndcg_at_k

0.00204697599657979

In [95]:
num_users = generated_matrix.shape[0]
k = 20
# Lista przechowująca wyniki NDCG dla każdego użytkownika
ndcg_results = []

# Oblicz NDCG dla każdego użytkownika
for user in range(num_users):
    ndcg_user = ndcg_score([original_small.iloc[user,:]], [generated_matrix.iloc[user,:]], k=20)
    ndcg_results.append(ndcg_user)

# Przekształć listę wyników NDCG na obiekt Series
ndcg_series = pd.Series(ndcg_results, name='NDCG@{}'.format(k))

# Utwórz DataFrame
df = pd.DataFrame({'user_id': range(1, num_users+1)})  # Zakładamy user_id jako 1, 2, ..., num_users

# Dołącz kolumnę z wynikami NDCG
df = pd.concat([df, ndcg_series], axis=1)

# Wydrukuj DataFrame z nową kolumną
print(df[df['NDCG@20'] > 0])

     user_id   NDCG@20
17        18  0.020238
32        33  0.005132
50        51  0.010990
75        76  0.013375
77        78  0.135820
88        89  0.009231
103      104  0.040217
107      108  0.026750
116      117  0.023031
142      143  0.027250
168      169  0.034627
180      181  0.040217


In [131]:
df[df['NDCG@20'] > 0]

,user_id,NDCG@20
17,18,0.020238
32,33,0.005132
50,51,0.010990
75,76,0.013375
77,78,0.135820
88,89,0.009231
103,104,0.040217
107,108,0.026750
116,117,0.023031
142,143,0.027250


In [47]:
generated.iloc[0,:]

1       12.748544
2       16.550229
3       14.956472
4       18.810884
5       26.120023
          ...    
1672     0.000986
1673    75.408967
1674     0.000986
1681     0.000740
1682     0.000740
Name: 0, Length: 1635, dtype: float64

In [138]:
d = original['user_id'].sort_values().to_list()

In [139]:
d

[24,
 31,
 32,
 34,
 40,
 45,
 50,
 61,
 64,
 66,
 67,
 68,
 71,
 73,
 77,
 78,
 79,
 87,
 97,
 108,
 110,
 111,
 121,
 137,
 138,
 140,
 142,
 159,
 166,
 169,
 175,
 193,
 199,
 200,
 209,
 210,
 211,
 214,
 216,
 219,
 236,
 240,
 248,
 251,
 255,
 260,
 261,
 262,
 266,
 267,
 276,
 281,
 282,
 287,
 303,
 306,
 307,
 308,
 310,
 311,
 312,
 324,
 327,
 328,
 332,
 333,
 335,
 352,
 356,
 360,
 364,
 371,
 372,
 381,
 383,
 389,
 395,
 405,
 410,
 413,
 423,
 425,
 426,
 430,
 434,
 437,
 445,
 448,
 449,
 450,
 452,
 468,
 469,
 478,
 480,
 482,
 498,
 500,
 507,
 508,
 517,
 522,
 523,
 524,
 527,
 529,
 533,
 535,
 539,
 542,
 543,
 552,
 560,
 569,
 573,
 586,
 588,
 589,
 591,
 596,
 599,
 606,
 607,
 612,
 614,
 616,
 618,
 622,
 626,
 627,
 639,
 650,
 657,
 661,
 670,
 674,
 676,
 681,
 686,
 693,
 695,
 704,
 705,
 706,
 708,
 714,
 724,
 729,
 731,
 732,
 745,
 755,
 756,
 765,
 766,
 771,
 775,
 779,
 781,
 785,
 794,
 803,
 811,
 818,
 823,
 826,
 830,
 833,
 847,
 849,

In [143]:
original.sort_values('user_id')

,user_id,1,2,3,4,5,6,7,8,9,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
3,24,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180,32,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
179,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,929,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
134,930,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
176,938,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
generated_matrix.sort_index()

NameError: name 'generated_matrix' is not defined

In [14]:
dictt = {'aa': [1,2,3], 'bb' : [14]}
for k in dictt.keys():
    print(dictt[k])

[1, 2, 3]
[14]


In [20]:
sum(dictt['aa']) / len(dictt['aa'])

2.0

In [ ]:
dict